## Аннотация: Рекомендация тарифов

На основе данных о поведении клиентов компании "Мегалайн", перешедших на тарифы "Смарт" и "Ультра" нужно построить модель для задачи классификации, которая выберет подходящий тариф (**цель** проекта). **Задача** проекта: выбрать модель с максимально большим значением *accuracy* (не менее 0.75).

## Знакомство с данными

In [5]:
#импортируем библиотеки, которые понадобятся нам в исследовании
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [6]:
initial_data = pd.read_csv('/datasets/users_behavior.csv') #применяем специальный метод для чтения файла формата csv
initial_data.info() #запрашиваем и выводим общую информацию об исходном датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [7]:
#произведем обзор полученных данных, вызвав 10 верхних и нижних строк датафрейма
display(initial_data.head(10))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [8]:
display(initial_data.tail(10))

,calls,minutes,messages,mb_used,is_ultra
3204,86.0,658.66,47.0,14153.10,0
3205,59.0,412.81,16.0,14105.03,0
3206,76.0,586.51,54.0,14345.74,0
3207,17.0,92.39,2.0,4299.25,0
3208,164.0,1016.98,71.0,17787.52,1
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


### Вывод:
Всего в датафрейме 5 столбцов и 3214  строк.
Пропусков нет, стилистических ошибок и проблем, требующих устранения, в данных не наблюдается, т.к. они уже были предобработаны в ходе более раннего исследования.

Перейдем к подготовке данных для обучения и использования моделей.

## Разделение данных на выборки

Нам нужно разбить данные на три части, получив в итоге обучающую, валидационную и тестовую выборки. Делить будем в соотношении 3:1:1, т.к. тестовая выборка "не спрятана", а также может быть выделена из исходных данных. 

Для этих целей применим дважды инструмент train_test_split. 

In [9]:
#выделяем признаки и целевой признак
features = initial_data.drop(['is_ultra'], axis=1) 
target = initial_data['is_ultra'] 

#отделяем от общего объема данных тестовую выборку
features, features_test, target, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)
#делим оставшиеся данные на обучающую и валидационную выборки
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

### Вывод:
На данном этапе мы подготовили исходные данные к использованию в обучении, применении и проверки работы моделей, а именно:
- выделили в отдельные переменные признаки и целевой признак (тариф, согласно обозначенной цели проекта);
- разделили данные на три выборки, которые будут использоваться на разных этапах работы с моделями.

Теперь можем перейти исследованию моделей.

## Исследование моделей

В данном исследовании мы будем работать с тремя моделями: решающее дерево, случайный лес и логистическая регрессия. На этом этапе мы обучим модели на тренировочной выборке и применим их на валидационной выборке, а также подсчитаем значение accuracy для определения качества модели (также на валидационной выборке). 

Начнем с решающего дерева. Построим цикл, который будет перебирать глубину дерева и сравнивать получаемые результаты для определения оптимального параметра и лучшего результата:

In [10]:
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0

for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) #обучаем модель на тренировочной выборке
    predictions_valid = model.predict(features_valid) #получаем предсказание на валидационной выборке
    result_tree = accuracy_score(target_valid, predictions_valid) #считаем качество модели на валидационной выборке
    if result_tree > best_tree_result:
        best_tree_model = model
        best_tree_result = result_tree
        best_tree_depth = depth
        
print('Лучшая глубина модели:', best_tree_depth)
print('Лучший результат решающего дерева:', best_tree_result.round(3))

Лучшая глубина модели: 7
Лучший результат решающего дерева: 0.774


Согласно подсчетам, лучший результат модели решающего дерева (0.774) был получен при глубине, равной 7. 

Перейдем к следующей модели - случайному лесу. Порядок действий будет аналогичным, за исключением того, что помимо глубины деревьев в лесу, мы будем также менять и их количество.

In [11]:
best_forest_model = None
best_forest_result = 0
best_forest_estimators = 0
best_forest_depth = 0

for est in range(1,101):
    for depth in range(1,21):
            model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
            model_forest.fit(features_train, target_train)
            predictions_valid = model_forest.predict(features_valid)
            result_forest = accuracy_score(target_valid, predictions_valid) 
            if result_forest > best_forest_result:
                best_forest_model = model_forest
                best_forest_result = result_forest
                best_forest_estimators = est
                best_forest_depth = depth
                
print('Лучшее количество деревьев:', best_forest_estimators)
print('Лучшая глубина модели:', best_forest_depth)
print('Лучший результат случайного леса:', best_forest_result.round(3))

Лучшее количество деревьев: 20
Лучшая глубина модели: 15
Лучший результат случайного леса: 0.802


Согласно подсчетам, лучший результат модели случайного леса (0.802) был получен при количестве деревьев, равном 20, и глубине, равной 15. 

Наконец перейдем к модели логистической регрессии. В ней мы не будем искать оптимальные параметры, поэтому необходимости в циклах также нет.

In [12]:
model_regression = LogisticRegression(random_state=12345)
model_regression.fit(features_train, target_train)
predictions_valid = model_regression.predict(features_valid)
result_regression = model.score(features_valid, target_valid) 
print('Результат логистической регрессии:', result_regression.round(3))

Результат логистической регрессии: 0.771


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Согласно подсчетам, лучший результат модели логистической регрессии - 0.771.

### Вывод:

Все исследованные модели преодолели пороговое значение accuracy. Наилучшую оценку показала модель случайного леса с 20-ю деревьями и глубиной, равной 15 - 0.802. Наихудший результат у наиболее быстрой и имеющей наименее громоздкий код логистической регрессии - 0.771. Впрочем, это всего на 3 тысячных меньше, чем у решающего дерева с глубиной, равной 7 (0.774).

На следующем этапе проверим исследованные модели на тестовой выборке.

## Проверка моделей на тестовой выборке

На данном этапе протестируем получившиеся и сохраненные на предыдущем шаге лучшие модели на тестовой выборке. Проверять будем в том же порядке, в каком модели обучались.

In [13]:
#тестируем решающее дерево
predictions_test = best_tree_model.predict(features_test)
test_result_tree = accuracy_score(target_test, predictions_test) 

#тестируем случайный лес
predictions_test = best_forest_model.predict(features_test)
test_result_forest = accuracy_score(target_test, predictions_test) 

#тестируем логистическую регрессию
predictions_test = model_regression.predict(features_test)
test_result_regression = accuracy_score(target_test, predictions_test) 

#выводим результаты на экран для сравнения
print('Accuracy решающего дерева:', test_result_tree.round(3))
print('Accuracy случайного леса:', test_result_forest.round(3))
print('Accuracy логистической регрессии:', test_result_regression.round(3))

Accuracy решающего дерева: 0.788
Accuracy случайного леса: 0.785
Accuracy логистической регрессии: 0.703


### Вывод:

При проверке на тестовой выборке лучший результат показала модель решающего дерева (0.788). Модель случайного леса уступила ей всего три тысячные (0.785), тогда как модель логистической регрессии не смогла преодолеть пороговое значение accuracy (0.703).

## Проверка моделей на адекватность

Теперь проверим наши модели на адекватность. Для этого сравним полученные на предыдущем шаге результаты с результатами константной модели. В библиотеке sklearn есть такая модель: DummyClassifier

In [14]:
dummy_clf = DummyClassifier(random_state=12345, strategy='most_frequent')
dummy_clf.fit(features_train, target_train)
result_dc = dummy_clf.score(features_test, target_test)
print('Результат проверки на адекватность:', result_dc.round(3))

Результат проверки на адекватность: 0.695


### Вывод:

Качество константной модели ниже выбранных нами лучших моделей, следовательно, проверку на адекватность они прошли. 

## Итоговый вывод

В данном проекте мы исследовали три модели машинного обучения: решающее дерево, случайный лес и логистическую регрессию. 

Обучив их на тренировочной выборке из исходных данных, мы применили их к валидационной выборке, подсчитали качество каждой модели, получив значения, выше порогового (0.75), а именно 0.774, 0.802 и 0.771 соответственно. 

При проверке на тестовой выборке качество моделей изменилось. У решающего дерева улучшилось (0.788), у случайного леса ухудшилось (0.785), у логистической регрессии ухудшилось так, что модель не смогла преодолеть пороговое значение (0.703).

Таким образом, можно сделать вывод, что наиболее подходящая для имеющихся данных модель - **решающее дерево**.